In [9]:
from Helperfunction import *
from Helper_miscelluous import *

(xy_loc_bulk, xy_loc_fl, xy_loc_xzz, yloc_bulk, yloc_fl, yloc_xzz, zloc_bulk, zloc_fl,
    zloc_xzz, off_diag_bulk_coe, off_diag_fl_coe, off_diag_xzz_coe, zloc_bulk_diag, zloc_fl_diag,
    zloc_xzz_diag, coe_bulk_diag, coe_fl_diag, coe_xzz_diag) = vmc_off_diag_es(16, 1, np.pi/2, True)

In [16]:
for i in zloc_bulk:
    
    print(i, zloc_bulk[i].shape)

(0, 3) (13, 0, 1)
(1, 2) (39, 1, 1)
(2, 1) (39, 2, 1)


In [29]:
dmrg_sample = jnp.load("../entanglement_swapping/DMRG/mps_tensors/sample_" + str(16) + "_angle_" + str(round(jnp.pi/2, 3)) + ".npy")

In [34]:
a = np.array([1, 2 ,3, 4, 5])
a[False]

array([], shape=(0, 5), dtype=int32)

In [3]:
import argparse
import itertools
import random
import os
import numpy as np
import optax
import jax
from jax import numpy as jnp
from Helperfunction import *
from Helper_miscelluous import *
from RNNfunction import *
import pickle
from jax import make_jaxpr
import jax.config
from jax.flatten_util import ravel_pytree

jax.config.update("jax_enable_x64", False)

parser = argparse.ArgumentParser()
parser.add_argument('--L', type = int, default = 8)
parser.add_argument('--p', type = int, default = 2)
parser.add_argument('--numunits', type = int, default=16)
parser.add_argument('--lr', type = float, default=1e-3)
parser.add_argument('--lrthreshold', type = float, default=5e-4)
parser.add_argument('--lrdecaytime', type = float, default=5000)
parser.add_argument('--mag_fixed', type = bool, default=False)
parser.add_argument('--Sz', type = int, default=0)
parser.add_argument('--gradient_clip', type = bool, default=True)
parser.add_argument('--gradient_clipvalue', type = float, default=10.0)
parser.add_argument('--dotraining', type = bool, default=True)
parser.add_argument('--T0', type = float, default= 0.0)
parser.add_argument('--Nwarmup', type = int, default=0)
parser.add_argument('--Nannealing', type = int, default=0) #10000
parser.add_argument('--Ntrain', type = int, default=0)
parser.add_argument('--Nconvergence', type = int, default=10000)
parser.add_argument('--numsamples', type = int, default=128)
parser.add_argument('--testing_sample', type = int, default=5e+4)
parser.add_argument('--lrthreshold_convergence', type = float, default=5e-4)
parser.add_argument('--lrdecaytime_convergence', type = float, default=2500)
parser.add_argument('--seed', type = int, default=3)
parser.add_argument('--rnn_type', type = str, default="tensor_gru")
parser.add_argument('--cmi_pattern', type = str, default="no_decay")
parser.add_argument('--sparsity', type = int, default=0)
parser.add_argument('--basis_rotation', type = bool, default=True)
parser.add_argument('--angle', type = float, default=0.000001)
parser.add_argument('--sr', type = bool, default=False)
args = parser.parse_args()

units = args.numunits
numsamples = args.numsamples
p = args.p
lr=args.lr
lrdecaytime = args.lrdecaytime
lrdecaytime_conv = args.lrdecaytime_convergence
lrthreshold = args.lrthreshold
lrthreshold_conv = args.lrthreshold_convergence
T0 = args.T0
mag_fixed = args.mag_fixed
magnetization = 2*args.Sz
gradient_clip = args.gradient_clip
gradient_clipvalue = args.gradient_clipvalue
dotraining = args.dotraining
Nwarmup = args.Nwarmup
Nannealing = args.Nannealing
Ntrain = args.Ntrain
Nconvergence = args.Nconvergence
numsteps = Nwarmup + (Nannealing+1)*Ntrain + Nconvergence
testing_sample = args.testing_sample
rnn_type = args.rnn_type
cmi_pattern = args.cmi_pattern
sparsity = args.sparsity
basis_rotation = args.basis_rotation
angle = args.angle
sr = args.sr
input_size = 2 ** p
L = args.L
key = PRNGKey(args.seed)
diag_bulk, diag_fl, diag_xzz =False, False, False
meanEnergy=[]
varEnergy=[]
m_list = []
N = L*p


for angle in (0.0*jnp.pi, 0.05*jnp.pi, 0.1*jnp.pi, 0.15*jnp.pi, 0.20*jnp.pi, 0.25*jnp.pi, 0.3*jnp.pi, 0.35*jnp.pi, 0.4*jnp.pi, 0.45*jnp.pi, 0.5*jnp.pi):
    # x and y are the cosine and sine of the rotation angle
    x, y = jnp.cos(angle), jnp.sin(angle)
    if (rnn_type == "tensor_gru"):
        params = init_tensor_gru_params(N, units, input_size, key)
    for i in range(L):
        m_list.append("tensor_"+str(i+1)+"_angle_"+str(angle)+".npy")
    # get the gradient function and compute cost is imported from Helper_miscelluous.py
    grad_f = jax.jit(jax.grad(compute_cost), static_argnums=(2,))
    optimizer = optax.adam(learning_rate=lr)
    optimizer_state = optimizer.init(params)
    fixed_params = N, p, units
    (xy_loc_bulk, xy_loc_fl, xy_loc_xzz, yloc_bulk, yloc_fl, yloc_xzz, zloc_bulk, zloc_fl,
    zloc_xzz, off_diag_bulk_coe, off_diag_fl_coe, off_diag_xzz_coe, zloc_bulk_diag, zloc_fl_diag,
    zloc_xzz_diag, coe_bulk_diag, coe_fl_diag, coe_xzz_diag) = vmc_off_diag_es(N, p, angle, basis_rotation)
    batch_sample_prob = jax.jit(vmap(sample_prob, (None, None, None, 0)), static_argnames=['fixed_params'])
    batch_log_amp = jax.jit(vmap(log_amp, (0, None, None, None)), static_argnames=['fixed_params'])
    batch_total_samples_1d = vmap(total_samples_1d, (0, None), 0)
    batch_new_coe_1d = vmap(new_coe_1d, (0, None, None, None, None))
    batch_diag_coe = vmap(diag_coe, (0, None, None, None, None, None, None))
    batch_log_phase_dmrg = jax.jit(vmap(log_phase_dmrg, (0, None, None, None)))
    wemb = jnp.eye(2**p)
    ang = round(angle, 3)
    M0 = jnp.load("../entanglement_swapping/DMRG/mps_tensors/tensor_init_"+str(N)+"_angle_"+str(ang)+".npy")
    M = jnp.load("../entanglement_swapping/DMRG/mps_tensors/tensor_"+str(N)+"_angle_"+str(ang)+".npy")
    Mlast = jnp.load("../entanglement_swapping/DMRG/mps_tensors/tensor_last_"+str(N)+"_angle_"+str(ang)+".npy")
    T = T0
    t = time.time()

    for it in range(0, numsteps):

        start = time.time()
        '''
        if it+1<=Nwarmup+Nannealing*Ntrain: #Before the end of annealing
            lr_adap = max(lrthreshold, lr/(1+it/lrdecaytime))
        elif it+1>Nwarmup+Nannealing*Ntrain: #After annealing -> finetuning the model during convergence
            lr_adap = lrthreshold_conv/(1+(it-(Nwarmup+Nannealing*Ntrain))/lrdecaytime_conv)
        '''
        key_ = split(key, numsamples)
        samples, sample_log_amp = batch_sample_prob(params, wemb, fixed_params, key_)
        sample_log_amp += batch_log_phase_dmrg(samples, M0, M, Mlast)
        samples_grad = samples.reshape(-1, N, p)
        key, subkey1, subkey2 = split(key_[0], 3)
        sigmas = jnp.concatenate((batch_total_samples_1d(samples, xy_loc_bulk),
                                 batch_total_samples_1d(samples, xy_loc_fl),
                                 batch_total_samples_1d(samples, xy_loc_xzz)), axis=1).reshape(-1, N, p)
        #minus sign account for the minus sign of each term in the Hamiltonian
        matrixelements = (jnp.concatenate((batch_new_coe_1d(samples, off_diag_bulk_coe, yloc_bulk, zloc_bulk, basis_rotation),
                                         batch_new_coe_1d(samples, off_diag_fl_coe, yloc_fl, zloc_fl, basis_rotation),
                                         batch_new_coe_1d(samples, off_diag_xzz_coe, yloc_xzz, zloc_xzz, basis_rotation)), axis=1)
                          .reshape(numsamples, -1))
        #print("matrixelements:", matrixelements)
        log_all_amp = batch_log_amp(sigmas, params, wemb, fixed_params) + batch_log_phase_dmrg(sigmas, M0, M, Mlast)
        log_diag_amp = jnp.repeat(sample_log_amp, (jnp.ones(numsamples)*(matrixelements.shape[1])).astype(int), axis=0)
        amp = jnp.exp(log_all_amp.ravel()-log_diag_amp).reshape(numsamples, -1)
        Eloc = jnp.sum((amp*matrixelements), axis=1) + batch_diag_coe(samples, zloc_bulk_diag, zloc_fl_diag, zloc_xzz_diag, coe_bulk_diag, coe_fl_diag, coe_xzz_diag)
        meanE,  varE = jnp.mean(Eloc), jnp.var(Eloc)

    

Flattened parameters: [1.  2.  3.  4.  5.  2.5 0.5 1.5]
Unflattened parameters: [Array([1., 2., 3.], dtype=float32), Array([4., 5.], dtype=float32), Array(2.5, dtype=float32), Array([0.5, 1.5], dtype=float32)]


In [78]:
import numpy as np
t1 = np.random.rand(1000000)
t2 = np.random.rand(1000000)*(1-t1)
t3 = np.random.rand(1000000)*(1-t1-t2)
t4 = 1-t1-t2-t3
p1 = np.random.rand(1000000)*2*np.pi
p2 = np.random.rand(1000000)*2*np.pi
p3 = np.random.rand(1000000)*2*np.pi
p4 = np.random.rand(1000000)*2*np.pi
x1 = np.sqrt(t1)*np.exp(1j*p1)
x2 = np.sqrt(t2)*np.exp(1j*p2)
x3 = np.sqrt(t3)*np.exp(1j*p3)
x4 = np.sqrt(t4)*np.exp(1j*p4)

In [79]:
np.sum(np.abs(x1)**2+np.abs(x2)**2+np.abs(x3)**2+np.abs(x4)**2)

1000000.0

In [80]:
px1 = np.abs(x1)**2
px2 = np.abs(x2)**2
px3 = np.abs(x3)**2
px4 = np.abs(x4)**2
px1x4 = px1+px4
px2x3 = px2+px3
C = 2*x1*x4-2*x2*x3
cp1 = (1+np.sqrt(1-np.abs(C)**2))/2
cp2 = (1-np.sqrt(1-np.abs(C)**2))/2
a1 = -px1*np.log(px1)-px2*np.log(px2)-px3*np.log(px3)-px4*np.log(px4)+px1x4*np.log(px1x4)+px2x3*np.log(px2x3)
b1 = -cp1*np.log(cp1)-cp2*np.log(cp2)

In [81]:
print(x1[636373], x2[636373], x3[636373], x4[636373])

(-0.6955850017902521+0.19609206259357648j) (0.05270686824463031-0.3134852381631305j) (-0.10491433363726148+0.5133506686988841j) (0.2637844027233604-0.18038919643270798j)


In [82]:
print(np.sum(a1>b1))

1000000


In [20]:
import numpy as np
a0 = np.random.rand(10000)
a1 = np.random.rand(10000)
a2 = np.random.rand(10000)
a3 = np.random.rand(10000)
x0 = a0**2+a1**2+a2**2+a3**2
x1 = np.sqrt((a0**2+a1**2-a2**2-a3**2)**2+4*(a0*a2+a1*a3)**2)
Sa = -a0*np.log(a0)-a1*np.log(a1)-a2*np.log(a2)-a3*np.log(a3)
Sx = -x0*np.log(x0)-x1*np.log(x1)

In [35]:
a = np.zeros((2, 2))

In [36]:
a[0:1, :].shape

(1, 2)